In [1]:
import re
import numpy as np
import pandas as pd
import requests
from lxml import etree 
from bs4 import BeautifulSoup
import json

In [2]:
res=requests.get('https://announce.fundclear.com.tw/MOPSonshoreFundWeb/B01_02.jsp',
                headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"})

if str(res) == '<Response [200]>':
    print('get web data successfully')
else:
    print('Failed to get web data')

# ----------------------------------------------------------------------------------------------
# 取得目標公司產品總覽

soup = BeautifulSoup(res.text, 'html.parser')


firm=[]
X1 = True
i = 0
while X1==True:
    for j in range(1,3):
        try:
            Firmlist=soup.select(f'.row{j}')[i]
            temp = {
                'ID': Firmlist.select('td')[0].get_text(),
                'Name': Firmlist.select('td')[1].get_text(),
            }
            firm.append(temp)
        except Exception as Err:
            X1=False
            print('firm data are all Done', Err)
    i+=1

#-----------------------------------------------------------------------------------------------------------------            
# 取得個據點資訊      
#post方式
target=['富蘭克林','安聯','聯博','摩根','元大','國泰']      

for n in target:
    for k in firm:
        if n in list(k.values())[1]:
            headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
            payload = {
                'type': 2,
                'agent': list(k.values())[0],
                'fundCom': 'ALL',
                'onClickB': 0
            }

            res=requests.post('https://announce.fundclear.com.tw/MOPSonshoreFundWeb/E03_01.jsp',headers=headers,data=payload)
            soup=BeautifulSoup(res.text,'html.parser')

            r=[]

            for i in soup.select('td'):
                r.append(str(i))

            columns=[]
            spotDict=[]
            spot=[]

            for j in r[44:]:
                try:
                    a=re.match(r'<td rowspan="\d{0,2}".*', j)
                    cat=a.group(0)
                    cat=cat.split('>')[1].split('<')[0]
                    columns.append(cat)
                except:
                    if 'align=' not in j:
                        spotDict.append({columns[-1]:j.split('>')[1].split('<')[0]})
                        spot.append(j.split('>')[1].split('<')[0])
                    else:
                        break   

            spot=list(set(spot))
            spot=sorted(spot)
            df=pd.DataFrame(index=spot,columns=columns)
            try:
                df=df.drop(index='\n')
            except:
                print(r'no index is \n anymore')

            for i in spotDict:
                df.at[list(i.values())[0],list(i.keys())[0]]=1

            try:
                df=df.drop(index='\n')
            except:
                print(r'no index is \n anymore')

            df=df.fillna(value=0)
            print(f'{n} is done') 
            df.to_csv(f'{n}spot.csv',encoding='utf-8')


get web data successfully
firm data are all Done list index out of range
富蘭克林 is done
                                   富蘭克林華美AI新科技基金  富蘭克林華美中國消費基金  \
三信商業銀行                                         1             1   
上海商業儲蓄銀行                                       1             1   
中國信託商業銀行                                       1             1   
中租投顧                                           1             1   
中華商業銀行(97年由香港商香港上海?豐銀行概括承受)                    0             0   
中華郵政股份有限公司                                     0             1   
京城商業銀行                                         1             1   
元大(原：元大寶來)證券股份有限公司                             1             1   
元大商業銀行                                         1             1   
元富證券股份有限公司                                     1             1   
兆豐國際商業銀行                                       1             1   
兆豐證券股份有限公司                                     1             1   
凱基(原:萬泰)商業銀行                                   0        

安聯 is done
                                   安聯DMAS全球傘型基金之安聯DMAS 50組合基金  \
三信商業銀行                                                      1   
上海商業儲蓄銀行                                                    1   
中國信託商業銀行                                                    1   
中租投顧                                                        0   
京城商業銀行                                                      1   
保證責任高雄市第三信用合作社                                              0   
元大(原：元大寶來)證券股份有限公司                                          1   
元大商業銀行                                                      1   
元富證券股份有限公司                                                  1   
兆豐國際商業銀行                                                    1   
兆豐證券股份有限公司                                                  1   
凱基(原:萬泰)商業銀行                                                1   
凱基證券股份有限公司                                                  1   
匯豐(台灣)商業銀行                                                  0   
台北富邦商業銀行      

聯博 is done
                                   金鼎科技金融基金  金鼎鼎盈基金  金鼎鼎益基金  金鼎雙高基金  匯達行動基金  \
上海商業儲蓄銀行                                  1       1       1       1       1   
中國信託商業銀行                                  1       0       1       0       1   
元大(原：元大寶來)證券股份有限公司                        0       0       0       0       0   
元大商業銀行                                    1       0       1       0       1   
元富證券股份有限公司                                0       0       0       0       0   
兆豐國際商業銀行                                  1       0       1       0       1   
凱基(原:萬泰)商業銀行                              1       1       1       1       1   
凱基證券股份有限公司                                0       0       0       0       0   
匯豐(台灣)商業銀行                                0       0       0       0       0   
台北富邦商業銀行                                  1       1       1       1       1   
台新國際商業銀行                                  0       0       0       0       0   
合作金庫銀行                                   

摩根 is done
                                   摩根大歐洲基金  摩根中小基金  摩根中美G2新勢力基金  摩根中國A股基金  \
三信商業銀行                                   1       1            1         1   
上海商業儲蓄銀行                                 1       1            1         1   
中國信託商業銀行                                 1       1            1         1   
中租投顧                                     1       1            0         1   
中華商業銀行(97年由香港商香港上海?豐銀行概括承受)              1       1            0         0   
京城商業銀行                                   1       1            1         1   
元大(原：元大寶來)證券股份有限公司                       1       1            1         1   
元大商業銀行                                   1       1            1         1   
元富證券股份有限公司                               1       1            1         1   
兆豐國際商業銀行                                 1       1            1         1   
凱基(原:萬泰)商業銀行                             1       1            0         1   
凱基證券股份有限公司                               1       1            1  

元大 is done
                 元大2001基金  元大20年期以上AAA至A級美元公司債券ETF基金  元大20年期以上BBB級美元公司債券ETF基金  \
三信商業銀行                  1                          0                        0   
上海商業儲蓄銀行                1                          0                        0   
中國信託商業銀行                1                          0                        0   
中國信託綜合證券股份有限公司          1                          0                        0   
中租投顧                    1                          0                        0   
...                   ...                        ...                      ...   
遠東國際商業銀行                1                          0                        0   
鉅亨投顧                    1                          0                        0   
陽信商業銀行                  1                          0                        0   
香港商香港上海匯豐銀行台北分行         1                          0                        0   
高雄銀行                    1                          0                        0   

                

In [141]:
firm

[{'ID': 'A0001', 'Name': '兆豐國際證券投資信託股份有限公司'},
 {'ID': 'A0003', 'Name': '第一金證券投資信託股份有限公司'},
 {'ID': 'A0004', 'Name': '匯豐中華證券投資信託股份有限公司'},
 {'ID': 'A0006', 'Name': '景順證券投資信託股份有限公司'},
 {'ID': 'A0007', 'Name': '瀚亞證券投資信託股份有限公司'},
 {'ID': 'A0008', 'Name': '保德信證券投資信託股份有限公司'},
 {'ID': 'A0009', 'Name': '統一證券投資信託股份有限公司'},
 {'ID': 'A0010', 'Name': '富邦證券投資信託股份有限公司'},
 {'ID': 'A0011', 'Name': '摩根證券投資信託股份有限公司'},
 {'ID': 'A0012', 'Name': '華南永昌證券投資信託股份有限公司'},
 {'ID': 'A0014', 'Name': '新光證券投資信託股份有限公司'},
 {'ID': 'A0015', 'Name': '瑞銀證券投資信託股份有限公司'},
 {'ID': 'A0016', 'Name': '群益證券投資信託股份有限公司'},
 {'ID': 'A0017', 'Name': '台中銀證券投資信託股份有公司'},
 {'ID': 'A0018', 'Name': '聯博證券投資信託股份有限公司'},
 {'ID': 'A0020', 'Name': '日盛證券投資信託股份有限公司'},
 {'ID': 'A0021', 'Name': '柏瑞證券投資信託股份有限公司'},
 {'ID': 'A0022', 'Name': '復華證券投資信託股份有限公司'},
 {'ID': 'A0025', 'Name': '永豐證券投資信託股份有限公司'},
 {'ID': 'A0027', 'Name': '宏利證券投資信託股份有限公司'},
 {'ID': 'A0031', 'Name': '貝萊德證券投資信託股份有限公司'},
 {'ID': 'A0032', 'Name': '野村證券投資信託股份有限公司'},
 {'ID': 'A0033', 'Name'

# function

In [146]:
# target=['富蘭克林','安聯','聯博','摩根','元大','國泰']
def fundspotcollect(*args):
    target=[]
    for num in args:
        target.append(num)
        
    import re
    import numpy as np
    import pandas as pd
    import requests
    from lxml import etree 
    from bs4 import BeautifulSoup
    import json

    res=requests.get('https://announce.fundclear.com.tw/MOPSonshoreFundWeb/B01_02.jsp',
                    headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"})

    if str(res) == '<Response [200]>':
        print('get web data successfully')
    else:
        print('Failed to get web data')

    soup = BeautifulSoup(res.text, 'html.parser')

    firm=[]
    X1 = True
    i = 0
    while X1==True:
        for j in range(1,3):
            try:
                Firmlist=soup.select(f'.row{j}')[i]
                temp = {
                    'ID': Firmlist.select('td')[0].get_text(),
                    'Name': Firmlist.select('td')[1].get_text(),
                }
                firm.append(temp)
            except Exception as Err:
                X1=False
                print('firm data are all Done', Err)
        i+=1

    for n in target:
        for k in firm:
            if n in list(k.values())[1]:
                headers = {"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
                payload = {
                    'type': 2,
                    'agent': list(k.values())[0],
                    'fundCom': 'ALL',
                    'onClickB': 0
                }

                res=requests.post('https://announce.fundclear.com.tw/MOPSonshoreFundWeb/E03_01.jsp',headers=headers,data=payload)
                soup=BeautifulSoup(res.text,'html.parser')

                r=[]

                for i in soup.select('td'):
                    r.append(str(i))

                columns=[]
                spotDict=[]
                spot=[]

                for j in r[44:]:
                    try:
                        a=re.match(r'<td rowspan="\d{0,2}".*', j)
                        cat=a.group(0)
                        cat=cat.split('>')[1].split('<')[0]
                        columns.append(cat)
                    except:
                        if 'align=' not in j:
                            spotDict.append({columns[-1]:j.split('>')[1].split('<')[0]})
                            spot.append(j.split('>')[1].split('<')[0])
                        else:
                            break   

                spot=list(set(spot))
                spot=sorted(spot)
                df=pd.DataFrame(index=spot,columns=columns)
                try:
                    df=df.drop(index='\n')
                except:
                    print(r'no index is \n anymore')

                for i in spotDict:
                    df.at[list(i.values())[0],list(i.keys())[0]]=1

                try:
                    df=df.drop(index='\n')
                except:
                    print(r'no index is \n anymore')

                df=df.fillna(value=0)
                print(f'{n} is done') 
                df.to_csv(f'{n}spot.csv',encoding='utf-8')




In [147]:
fundspotcollect('富蘭克林','安聯','聯博','摩根','元大','國泰')

get web data successfully
firm data are all Done list index out of range
富蘭克林 is done
安聯 is done
聯博 is done
摩根 is done
元大 is done
國泰 is done


In [133]:
columns

['元大2001基金',
 '元大20年期以上AAA至A級美元公司債券ETF基金',
 '元大20年期以上BBB級美元公司債券ETF基金',
 '元大MSCI中國A股國際通ETF基金',
 '元大人民幣貨幣市場基金',
 '元大大中華TMT基金',
 '元大大中華價值指數基金',
 '元大大中華豐益平衡基金',
 '元大上證50基金',
 '元大中國平衡基金',
 '元大中國高收益點心債券基金',
 '元大中國國債及政策性金融債3至5年期債券ETF基金',
 '元大中國機會債券基金',
 '元大巴西指數基金',
 '元大巴菲特基金',
 '元大台商收成基金',
 '元大台灣50單日反向1倍基金(本基金為策略交易型產品且投資具有反向操作風險，故本基金不適合長期持有，僅符合臺灣證券交易所訂適格條件之投資人始得交易)',
 '元大台灣50單日正向2倍基金(本基金為策略交易型產品且投資具有槓桿操作風險，故本基金不適合長期持有，僅符合臺灣證券交易所訂適格條件之投資人始得交易)',
 '元大台灣中型100基金',
 '元大台灣加權股價指數基金',
 '元大台灣卓越50ETF連結基金',
 '元大台灣卓越50基金',
 '元大台灣金融基金',
 '元大台灣高股息ETF連結基金',
 '元大台灣高股息低波動ETF基金',
 '元大台灣高股息優質龍頭基金',
 '元大台灣電子科技基金',
 '元大平衡基金',
 '元大全球ETF成長組合基金',
 '元大全球ETF穩健組合基金',
 '元大全球人工智慧ETF基金',
 '元大全球不動產證券化基金',
 '元大全球公用能源效率基金',
 '元大全球平衡基金',
 '元大全球未來關鍵科技ETF基金',
 '元大全球地產建設入息基金',
 '元大全球股票入息基金',
 '元大全球美元公司債券基金',
 '元大全球國富債券基金',
 '元大全球債券基金',
 '元大全球新能源基金',
 '元大全球新興市場精選組合基金',
 '元大全球資源傘型基金之元大全球礦金基金',
 '元大全球農業商機基金',
 '元大全球滿意入息基金',
 '元大全球靈活配置債券組合基金',
 '元大多元基金',
 '元大多多基金',
 '元大多利基金',
 '元大多福基金',
 '元大印尼指數基金',
 '元大印度指數基金',
 '元大印度基金',
 '元大亞太成長基

In [134]:
spotDict

[{'元大2001基金': '元大(原：元大寶來)證券股份有限公司'},
 {'元大2001基金': '日盛證券股份有限公司'},
 {'元大2001基金': '元富證券股份有限公司'},
 {'元大2001基金': '兆豐證券股份有限公司'},
 {'元大2001基金': '凱基證券股份有限公司'},
 {'元大2001基金': '台新綜合證券股份有限公司'},
 {'元大2001基金': '合作金庫證券股份有限公司'},
 {'元大2001基金': '國票綜合證券股份有限公司'},
 {'元大2001基金': '國泰綜合證券股份有限公司'},
 {'元大2001基金': '基富通證券股份有限公司'},
 {'元大2001基金': '大昌證券股份有限公司'},
 {'元大2001基金': '富邦綜合證券股份有限公司'},
 {'元大2001基金': '康和綜合證券股份有限公司'},
 {'元大2001基金': '日盛國際商業銀行'},
 {'元大2001基金': '丹尼爾投顧'},
 {'元大2001基金': '勤美投顧(已退會)'},
 {'元大2001基金': '安睿宏觀投顧'},
 {'元大2001基金': '容海國際投顧'},
 {'元大2001基金': '富裕環球投顧'},
 {'元大2001基金': '廣源投顧'},
 {'元大2001基金': '永鈺投顧'},
 {'元大2001基金': '鉅亨投顧'},
 {'元大2001基金': '三信商業銀行'},
 {'元大2001基金': '上海商業儲蓄銀行'},
 {'元大2001基金': '中國信託商業銀行'},
 {'元大2001基金': '中華商業銀行(97年由香港商香港上海?豐銀行概括承受)'},
 {'元大2001基金': '中華郵政股份有限公司'},
 {'元大2001基金': '京城商業銀行'},
 {'元大2001基金': '保證責任高雄市第三信用合作社'},
 {'元大2001基金': '元大商業銀行'},
 {'元大2001基金': '兆豐國際商業銀行'},
 {'元大2001基金': '凱基(原:萬泰)商業銀行'},
 {'元大2001基金': '匯豐(台灣)商業銀行'},
 {'元大2001基金': '台北富邦商業銀行'},
 {'元大2001基金': '台新國際商業銀行'},
 

In [135]:
spot

['元大(原：元大寶來)證券股份有限公司',
 '日盛證券股份有限公司',
 '元富證券股份有限公司',
 '兆豐證券股份有限公司',
 '凱基證券股份有限公司',
 '台新綜合證券股份有限公司',
 '合作金庫證券股份有限公司',
 '國票綜合證券股份有限公司',
 '國泰綜合證券股份有限公司',
 '基富通證券股份有限公司',
 '大昌證券股份有限公司',
 '富邦綜合證券股份有限公司',
 '康和綜合證券股份有限公司',
 '日盛國際商業銀行',
 '丹尼爾投顧',
 '勤美投顧(已退會)',
 '安睿宏觀投顧',
 '容海國際投顧',
 '富裕環球投顧',
 '廣源投顧',
 '永鈺投顧',
 '鉅亨投顧',
 '三信商業銀行',
 '上海商業儲蓄銀行',
 '中國信託商業銀行',
 '中華商業銀行(97年由香港商香港上海?豐銀行概括承受)',
 '中華郵政股份有限公司',
 '京城商業銀行',
 '保證責任高雄市第三信用合作社',
 '元大商業銀行',
 '兆豐國際商業銀行',
 '凱基(原:萬泰)商業銀行',
 '匯豐(台灣)商業銀行',
 '台北富邦商業銀行',
 '台新國際商業銀行',
 '合作金庫銀行',
 '國泰世華商業銀行',
 '安泰商業銀行',
 '彰化商業銀行',
 '德商德意志銀行台北分行',
 '慶豐商業銀行(99年由元大銀行及遠東銀行概括承受)',
 '中租投顧',
 '永豐金(原：建華)證券股份有限公司',
 '玉山綜合證券股份有限公司',
 '第一金(原：一銀)證券股份有限公司',
 '統一綜合證券股份有限公司',
 '群益金鼎證券股份有限公司',
 '臺銀綜合證券股份有限公司',
 '華南永昌綜合證券股份有限公司',
 '台証期貨股份有限公司(98年併入凱基期貨)',
 '星展(台灣)商業銀行',
 '有限責任淡水第一信用合作社',
 '有限責任台中市第二信用合作社',
 '板信商業銀行',
 '永豐商業銀行',
 '法商法國巴黎銀行台北分行',
 '渣打國際商業銀行(96年更名並受讓英商渣打銀行在台全部營業及資產)',
 '澳商澳盛銀行集團股份有限公司(102年分割予澳盛(台灣)銀行)',
 '玉山商業銀行',
 '王道商業銀行股份有限公司',
 '瑞興(原:大台北/稻江)商業銀行',
 '瑞士商瑞士銀行台北分行',
 '第

In [144]:
df

,國泰15年期以上A等級醫療保健產業債券基金,國泰A50傘型基金之富時中國A50單日反向1倍基金,國泰A50傘型基金之富時中國A50單日正向2倍基金,國泰人民幣貨幣市場基金,國泰大中華基金,國泰小龍基金,國泰中小成長基金,國泰中國內需增長基金,國泰中國企業7年期以上美元A級債券基金,國泰中國高收益債券基金,...,國泰翔鷹債券基金,國泰新興市場基金,國泰新興高收益債券基金,國泰道瓊工業平均指數單日正向2倍基金,國泰精選平衡基金,國泰網路資安ETF基金,國泰價值卓越基金,國泰歐洲精選基金,國泰豐利平衡基金,國泰豐益債券組合基金
上海商業儲蓄銀行,0,0,0,1,1,1,1,1,0,1,...,0,1,1,0,1,0,0,1,1,0
中國信託商業銀行,1,0,0,1,1,1,1,1,0,1,...,0,1,1,1,1,0,1,1,1,1
中國信託綜合證券股份有限公司,0,0,0,1,1,1,1,1,0,1,...,0,1,1,0,0,0,1,1,0,1
中租投顧,0,0,0,1,1,1,1,1,0,1,...,0,1,1,0,0,0,1,1,0,1
中華郵政股份有限公司,0,1,1,1,1,1,1,0,0,1,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
華南商業銀行,0,0,0,1,1,1,1,1,0,1,...,0,1,1,0,1,0,0,0,1,1
華南永昌綜合證券股份有限公司,0,1,1,1,1,1,1,1,0,1,...,0,1,1,1,1,1,1,1,1,1
華泰商業銀行,0,0,0,1,1,1,1,1,0,1,...,0,1,1,0,1,0,1,1,1,1
鉅亨投顧,0,0,0,1,1,1,1,1,0,1,...,0,1,1,0,0,0,1,1,0,1
